<a href="https://colab.research.google.com/github/Wiger123/outlier_quant/blob/main/hedge_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 强平价计算

In [1]:
maint_lookup_table = [
  (50_000, 0.4, 0),
  (250_000, 0.5, 50),
  (1_000_000, 1.0, 1_300),
  (10_000_000, 2.5, 16_300),
  (20_000_000, 5.0, 266_300),
  (50_000_000, 10.0, 1_266_300),
  (100_000_000, 12.5, 2_516_300),
  (200_000_000, 15.0, 5_016_300),
  (300_000_000, 25.0, 25_016_300),
  (500_000_000, 50.0, 100_016_300)
]


def binance_btc_liq_balance(wallet_balance, contract_qty, entry_price):
  for max_position, maint_margin_rate_pct, maint_amount in maint_lookup_table:
    maint_margin_rate = maint_margin_rate_pct / 100
    liq_price = (wallet_balance + maint_amount - contract_qty*entry_price) / (abs(contract_qty) * (maint_margin_rate - (1 if contract_qty>=0 else -1)))
    base_balance = liq_price * abs(contract_qty)
    if base_balance <= max_position:
      break
  return liq_price


def binance_btc_liq_leverage(leverage, contract_qty, entry_price):
  wallet_balance = abs(contract_qty) * entry_price / leverage
  return binance_btc_liq_balance(wallet_balance, contract_qty, entry_price)

### 边缘价格

In [2]:
def long_liquidation_price(lev, qty, ent, delta):
  return binance_btc_liq_leverage(leverage=lev, contract_qty=qty, entry_price=ent) + delta


def short_liquidation_price(lev, qty, ent, delta):
  return binance_btc_liq_leverage(leverage=lev, contract_qty=-qty, entry_price=ent) - delta

### 测试边缘价格

In [3]:
test_long_list = [
  (50, 0.510, 49019.99, 48242.37),
  (50, 0.310, 47755.10, 46994.77),
  (25, 0.010, 48259.20, 46521.13),
  (50, 0.050, 48975.01, 48203.92),
  (50, 0.051, 49334.51, 48555.74),
  (20, 0.199, 3192.00, 3048.28),
  (10, 350.4, 8.2055, 7.4612)
]

test_short_list = [
  (50, 0.509, 49036.12, 49807.80),
  (25, 0.010, 48259.64, 49986.87),
  (50, 0.050, 48989.20, 49750.39),
  (50, 0.150, 49429.99, 50206.25),
  (50, 0.051, 49353.69, 50120.99),
  (20, 0.199, 3199.00, 3341.60),
  (50, 0.426, 48073.33, 48833.46)
]

def test_calc_result():
  # 多单测试
  print("==========多单==========")
  for lev, q, entry, liquid in test_long_list:
    liquid_calc = long_liquidation_price(lev, q, entry, 50)
    print("计算强平价:{0}, 实际强平价:{1}, 差值:{2}, 是否合理:{3}, 差值比例:{4}%".format(round(liquid_calc, 2), liquid, round(liquid-liquid_calc, 2), liquid<liquid_calc, round((liquid-liquid_calc)/liquid*100, 5)))

  # 空单测试
  print("==========空单==========")
  for lev, q, entry, liquid in test_short_list:
    liquid_calc = short_liquidation_price(lev, q, entry, 50)
    print("计算强平价:{0}, 实际强平价:{1}, 差值:{2}, 是否合理:{3}, 差值比例:{4}%".format(round(liquid_calc, 2), liquid, round(liquid-liquid_calc, 2), liquid>liquid_calc, round((liquid-liquid_calc)/liquid*100, 5)))

In [4]:
test_calc_result()

==========多单==========
计算强平价:48282.52, 实际强平价:48242.37, 差值:-40.15, 是否合理:True, 差值比例:-0.08323%
计算强平价:47037.95, 实际强平价:46994.77, 差值:-43.18, 是否合理:True, 差值比例:-0.09188%
计算强平价:46564.89, 实际强平价:46521.13, 差值:-43.76, 是否合理:True, 差值比例:-0.09407%
计算强平价:48238.26, 实际强平价:48203.92, 差值:-34.34, 是否合理:True, 差值比例:-0.07124%
计算强平价:48591.99, 实际强平价:48555.74, 差值:-36.25, 是否合理:True, 差值比例:-0.07465%
计算强平价:3094.58, 实际强平价:3048.28, 差值:-46.3, 是否合理:True, 差值比例:-1.51883%
计算强平价:57.41, 实际强平价:7.4612, 差值:-49.95, 是否合理:True, 差值比例:-669.50904%
==========空单==========
计算强平价:49767.57, 实际强平价:49807.8, 差值:40.23, 是否合理:True, 差值比例:0.08077%
计算强平价:49940.07, 实际强平价:49986.87, 差值:46.8, 是否合理:True, 差值比例:0.09363%
计算强平价:49719.9, 实际强平价:49750.39, 差值:30.49, 是否合理:True, 差值比例:0.06128%
计算强平价:50167.72, 实际强平价:50206.25, 差值:38.53, 是否合理:True, 差值比例:0.07675%
计算强平价:50090.2, 实际强平价:50120.99, 差值:30.79, 是否合理:True, 差值比例:0.06143%
计算强平价:3295.57, 实际强平价:3341.6, 差值:46.03, 是否合理:True, 差值比例:1.37755%
计算强平价:48789.44, 实际强平价:48833.46, 差值:44.02, 是否合理:True, 差值比例:0.09015%


### 平均开仓价格

In [5]:
def average_entry_price(prc_a, qty_a, prc_b, qty_b):
  return (prc_a * qty_a + prc_b * qty_b) / (qty_a + qty_b)

### 测试平均开仓价格

In [6]:
print("==========均价==========")
print(average_entry_price(10, 2, 20, 3))

==========均价==========
16.0


### 计算可开量

In [7]:
def get_quantity(blc, lev, prc, ratio=0.99):
  return blc * lev / prc * ratio

### 测试可开量

In [8]:
print("==========可开量==========")
print(get_quantity(249.99, 25, 49839))

==========可开量==========
0.12414479624390538


### 网格挂单

In [26]:
balance = 10000
ratio_list = [3, 3, 6, 12, 24]
leverage = 30
mark_price = 49423.33
delta = 60

In [27]:
def get_price_list(balance, ratio_list, leverage, mark_price, dec_price, dec_quantity):
  long_price_list = []
  long_quant_list = []
  short_price_list = []
  short_quant_list = []

  qty = get_quantity(balance, leverage, mark_price)
  qty_long = 0
  qty_short = 0
  last_avg_prc_long = mark_price
  last_avg_prc_short = mark_price

  for i in range(len(ratio_list)):
    # 1. added quantity
    long_q = qty * ratio_list[i] / 100
    long_quant_list.append(round(long_q, dec_quantity))
    # 2. last avg price and last liquid price
    if i == 0:
      long_p = mark_price
    else:
      long_p = long_liquidation_price(leverage, qty_long, last_avg_prc_long, delta)
    # 3. append last liquid price
    long_price_list.append(round(long_p, dec_price))
    # 4. update last avg entry price
    last_avg_prc_long = average_entry_price(long_p, qty_long, last_avg_prc_long, long_q)
    # 5. update quantity
    qty_long += long_q

    # 1. added quantity
    short_q = qty * ratio_list[i] / 100
    short_quant_list.append(round(short_q, dec_quantity))
    # 2. last avg price and last liquid price
    if i == 0:
      short_p = mark_price
    else:
      short_p = short_liquidation_price(leverage, qty_short, last_avg_prc_short, delta)
    # 3. append last liquid price
    short_price_list.append(round(short_p, dec_price))
    # 4. update last avg entry price
    last_avg_prc_short = average_entry_price(short_p, qty_short, last_avg_prc_short, short_q)
    # 5. update quantity
    qty_short += short_q
  
  return long_price_list, long_quant_list, short_price_list, short_quant_list

### 测试网格挂单

In [28]:
long_p, long_q, short_p, short_q = get_price_list(balance, ratio_list, leverage, mark_price, 2, 3)
print('\n', long_p, '\n', long_q, '\n', short_p, '\n', short_q)


 [49423.33, 48027.76, 47350.52, 46683.26, 46037.17] 
 [0.18, 0.18, 0.361, 0.721, 1.442] 
 [49423.33, 50807.31, 51519.51, 52242.12, 52957.01] 
 [0.18, 0.18, 0.361, 0.721, 1.442]
